In [ ]:
#!/usr/bin/env python3
"""
Hadoop & MapReduce Student Hands-On Demo Script
Module 10: Scalable Computing - From Single Machine to Distributed Processing

This script demonstrates MapReduce concepts using Python and PySpark,
bridging the gap between the Java lab and practical data science workflows.

Usage:
    python student_hadoop_demo.py

Prerequisites:
    pip install pyspark pandas numpy matplotlib seaborn

Terminal commands to run before starting:
    # 1. Check Python version
    python --version

    # 2. Install required packages
    pip install pyspark pandas numpy matplotlib seaborn

    # 3. Download Titanic dataset
    curl -o titanic.csv https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv

    # 4. Verify Java installation (required for PySpark)
    java -version
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import sys
from typing import List, Tuple, Dict, Any

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")


In [ ]:
def print_header(title: str):
    """Print a formatted header"""
    print(f"\n{'='*60}")
    print(f"🚀 {title}")
    print(f"{'='*60}")


### Environment Check

Let's make sure all packages are installed.

In [ ]:
def check_environment():
    """Check if all required packages are available"""
    print_header("Environment Check")

    # Check Python version
    print(f"Python version: {sys.version}")

    # Test basic imports
    try:
        import pandas as pd
        import numpy as np
        import matplotlib.pyplot as plt
        import seaborn as sns
        print("✅ Core data science libraries imported successfully")
    except ImportError as e:
        print(f"❌ Import error: {e}")
        print("Run: pip install pandas numpy matplotlib seaborn")
        return False

    # Test PySpark
    try:
        from pyspark.sql import SparkSession
        print("✅ PySpark imported successfully")
    except ImportError as e:
        print(f"❌ PySpark import error: {e}")
        print("Run: pip install pyspark")
        return False

    print("\n🎯 Ready to start the Hadoop demo!")
    return True


### Load Data

We'll use Titanic data to demonstrate concepts.

In [ ]:
def load_titanic_data() -> pd.DataFrame:
    """Load Titanic dataset with fallback to generated data"""
    try:
        df = pd.read_csv('titanic.csv')
        print(f"✅ Titanic dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
        print(".2f")
        return df
    except FileNotFoundError:
        print("❌ titanic.csv not found. Download it first:")
        print("curl -o titanic.csv https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
        print("📝 Using generated sample data for demo")

        # Create sample data for demo
        np.random.seed(42)
        df = pd.DataFrame({
            'PassengerId': range(1, 892),
            'Survived': np.random.choice([0, 1], 891),
            'Pclass': np.random.choice([1, 2, 3], 891),
            'Name': [f'Passenger_{i}' for i in range(891)],
            'Sex': np.random.choice(['male', 'female'], 891),
            'Age': np.random.normal(30, 15, 891).clip(0, 80),
            'Fare': np.random.exponential(30, 891)
        })
        return df


## Part 1: Why Distributed Computing?

Single computers can't handle huge datasets. We need to distribute work across multiple machines.

In [ ]:
def demonstrate_single_machine_limits(df: pd.DataFrame):
    """Show why single machines can't handle big data"""
    print_header("Part 1: Single Machine Limitations")

    # Show basic analysis
    print("📋 Basic Analysis:")
    survival_rate = df['Survived'].mean()
    print(".1%")

    # Survival by class
    survival_by_class = df.groupby('Pclass')['Survived'].mean()
    print("\n🎯 Survival by Passenger Class:")
    for pclass, rate in survival_by_class.items():
        print(".1%")

    # Simulate "big data" problem
    print("\n💭 What if we had 1 BILLION passengers?")
    big_data_rows = 1_000_000_000
    estimated_memory = (df.memory_usage(deep=True).sum() / len(df)) * big_data_rows / 1024 / 1024 / 1024
    print(".1f")
    print("   Processing time would be impractical!")
    print("   → This is why we need distributed computing!")


## Part 2: Manual MapReduce

MapReduce has 3 phases:
1. **Map**: Extract key-value pairs
2. **Shuffle**: Group by key
3. **Reduce**: Aggregate results

In [ ]:
def manual_mapreduce_demo(df: pd.DataFrame) -> Dict[int, Tuple[int, float, int, int]]:
    """Demonstrate manual MapReduce implementation"""
    print_header("Part 2: Manual MapReduce Implementation")

    # MAP Phase: Extract survival data by class
    def mapper_survival_by_class(record) -> Tuple[int, int]:
        """Map function: extract (pclass, survived) pairs"""
        pclass = record['Pclass']
        survived = record['Survived']
        return (pclass, survived)

    print("🔹 MAP PHASE: Extracting key-value pairs...")
    mapped_data = []
    for _, row in df.iterrows():
        mapped_data.append(mapper_survival_by_class(row))

    print(f"   Generated {len(mapped_data)} key-value pairs")
    print(f"   Sample pairs: {mapped_data[:5]}")

    # SHUFFLE Phase: Group by key
    def shuffle_group_by_key(mapped_data: List[Tuple[int, int]]) -> Dict[int, List[int]]:
        """Shuffle function: group values by key"""
        shuffled = {}
        for key, value in mapped_data:
            if key not in shuffled:
                shuffled[key] = []
            shuffled[key].append(value)
        return shuffled

    print("\n🔀 SHUFFLE PHASE: Grouping by passenger class...")
    shuffled_data = shuffle_group_by_key(mapped_data)
    for key, values in shuffled_data.items():
        print(f"   Class {key}: {len(values)} passengers, survival indicators: {values[:5]}...")

    # REDUCE Phase: Calculate survival rates
    def reducer_survival_rate(key: int, values: List[int]) -> Tuple[int, float, int, int]:
        """Reduce function: calculate survival statistics"""
        total_passengers = len(values)
        survived_count = sum(values)
        survival_rate = survived_count / total_passengers
        return (key, survival_rate, survived_count, total_passengers)

    print("\n⚡ REDUCE PHASE: Calculating survival rates...")
    final_results = {}
    for key, values in shuffled_data.items():
        result = reducer_survival_rate(key, values)
        final_results[key] = result
        pclass, rate, survived, total = result
        print(".1%")

    print("\n🎉 Manual MapReduce completed!")
    print("💡 Key insight: Each phase can run in parallel across different machines!")

    return final_results


## Part 3: PySpark

Spark makes MapReduce much easier with high-level APIs.

In [ ]:
def pyspark_demo(df: pd.DataFrame, manual_results: Dict[int, Tuple[int, float, int, int]]):
    """Demonstrate PySpark MapReduce implementation"""
    print_header("Part 3: PySpark MapReduce Implementation")

    from pyspark.sql import SparkSession

    # Initialize Spark Session
    print("🚀 Initializing Spark Session...")
    try:
        spark = SparkSession.builder \
            .appName("TitanicMapReduceDemo") \
            .config("spark.sql.adaptive.enabled", "true") \
            .config("spark.driver.host", "localhost") \
            .config("spark.ui.enabled", "false") \
            .master("local[*]") \
            .getOrCreate()
    except Exception as e:
        print(f"❌ Spark initialization failed: {e}")
        print("💡 This is expected in some environments. Let's continue with pandas-only demo.")
        return None

    print(f"✅ Spark {spark.version} session created")

    # Load data into Spark DataFrame
    print("\n📥 Loading Titanic data into Spark...")
    spark_df = spark.createDataFrame(df)
    print(f"✅ Spark DataFrame created: {spark_df.count()} rows")

    # Show schema
    print("\n📋 Data Schema:")
    spark_df.printSchema()

    # Spark SQL approach (equivalent to HiveQL)
    print("\n🔍 Spark SQL Query (equivalent to HiveQL):")
    spark_df.createOrReplaceTempView("titanic")

    sql_result = spark.sql("""
        SELECT
            Pclass,
            AVG(Survived) as survival_rate,
            COUNT(*) as total_passengers,
            SUM(Survived) as survived_count
        FROM titanic
        GROUP BY Pclass
        ORDER BY Pclass
    """)

    print("📊 Results from Spark SQL:")
    sql_result.show()

    # Convert to Pandas for comparison
    spark_results = sql_result.toPandas()
    print("\n🔄 Comparison with our manual MapReduce:")
    for _, row in spark_results.iterrows():
        manual_result = manual_results.get(row['Pclass'], (0, 0, 0, 0))
        manual_rate = manual_result[1]
        spark_rate = row['survival_rate']
        diff = abs(manual_rate - spark_rate)
        print(".3%")

    print("\n🎯 Spark automatically handles the MapReduce complexity!")
    print("💡 This is how Hadoop/Hive works under the hood.")

    # Cleanup
    spark.stop()
    print("\n🧹 Spark session stopped.")

    return spark_results


### Spark Session

Initialize Spark to start processing.

In [ ]:
def performance_comparison(df: pd.DataFrame):
    """Compare performance between pandas and Spark"""
    print_header("Part 4: Performance Analysis")

    from pyspark.sql import SparkSession

    # Initialize Spark
    spark = SparkSession.builder \
        .appName("PerformanceTest") \
        .config("spark.sql.adaptive.enabled", "true") \
        .getOrCreate()

    spark_df = spark.createDataFrame(df)

    # Test 1: Pandas performance
    print("🐼 Testing Pandas Performance...")
    start_time = time.time()
    pandas_result = df.groupby('Pclass')['Survived'].agg(['mean', 'count', 'sum'])
    pandas_time = time.time() - start_time
    print(".4f")

    # Test 2: Spark performance
    print("\n⚡ Testing Spark Performance...")
    start_time = time.time()
    spark_result = spark_df.groupBy("Pclass") \
        .agg({"Survived": "avg", "PassengerId": "count"}) \
        .withColumnRenamed("avg(Survived)", "survival_rate") \
        .withColumnRenamed("count(PassengerId)", "total_passengers")
    spark_time = time.time() - start_time
    print(".4f")

    # Performance comparison
    speedup = pandas_time / spark_time if spark_time > 0 else float('inf')
    print("\n🏁 Performance Results:")
    print(".4f")
    print(".4f")
    print(".2f")
    print("\n💡 Why Spark might be slower on small data:")
    print("   - Overhead of distributed processing")
    print("   - Data serialization/deserialization")
    print("   - But Spark scales much better with big data!")

    # Cleanup
    spark.stop()

    return pandas_result, pandas_time, spark_time


## Part 5: Visualizations

Create charts to understand our results.

In [ ]:
def create_visualizations(df: pd.DataFrame, manual_results: Dict, spark_results):
    """Create comparison visualizations"""
    print_header("Part 5: Visual Analysis")

    # Create comparison plots
    if spark_results is None:
        # Simplified visualization without Spark comparison
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))

        # Plot 1: Manual MapReduce results
        classes = sorted(manual_results.keys())
        survival_rates = [manual_results[k][1] for k in classes]
        axes[0].bar(classes, survival_rates, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.7)
        axes[0].set_title('Survival Rate by Class\n(Manual MapReduce)')
        axes[0].set_ylabel('Survival Rate')
        axes[0].set_xticks(classes)

        # Plot 2: Survival by gender and class
        gender_class = df.groupby(['Pclass', 'Sex'])['Survived'].mean().unstack()
        gender_class.plot(kind='bar', ax=axes[1], alpha=0.7)
        axes[1].set_title('Survival Rate by Class and Gender')
        axes[1].set_ylabel('Survival Rate')
        axes[1].legend(title='Gender')

        print("📊 Note: Spark comparison skipped due to environment limitations")
    else:
        # Full comparison if Spark works
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))

        # Plot 1: Manual MapReduce results
        classes = sorted(manual_results.keys())
        survival_rates = [manual_results[k][1] for k in classes]
        axes[0, 0].bar(classes, survival_rates, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.7)
        axes[0, 0].set_title('Survival Rate by Class\n(Manual MapReduce)')
        axes[0, 0].set_ylabel('Survival Rate')
        axes[0, 0].set_xticks(classes)

        # Plot 2: Spark SQL results
        spark_rates = spark_results.set_index('Pclass')['survival_rate']
        axes[0, 1].bar(spark_rates.index, spark_rates.values, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.7)
        axes[0, 1].set_title('Survival Rate by Class\n(Spark SQL)')
        axes[0, 1].set_ylabel('Survival Rate')
        axes[0, 1].set_xticks(classes)

        # Plot 3: Difference between methods
        differences = []
        for pclass in classes:
            manual_rate = manual_results[pclass][1]
            spark_rate = spark_results[spark_results['Pclass'] == pclass]['survival_rate'].iloc[0]
            differences.append(abs(manual_rate - spark_rate))

        axes[1, 0].bar(classes, differences, color='orange', alpha=0.7)
        axes[1, 0].set_title('Difference Between Methods')
        axes[1, 0].set_ylabel('Absolute Difference')
        axes[1, 0].set_xticks(classes)

        # Plot 4: Survival by gender and class
        gender_class = df.groupby(['Pclass', 'Sex'])['Survived'].mean().unstack()
        gender_class.plot(kind='bar', ax=axes[1, 1], alpha=0.7)
        axes[1, 1].set_title('Survival Rate by Class and Gender')
        axes[1, 1].set_ylabel('Survival Rate')
        axes[1, 1].legend(title='Gender')

    plt.tight_layout()
    plt.savefig('student_hadoop_demo_results.png', dpi=150, bbox_inches='tight')
    print("📊 Visualizations saved as 'student_hadoop_demo_results.png'")
    plt.show()


## Part 6: Java Connection

See how this connects to your Java MapReduce lab.

In [ ]:
def demonstrate_java_connection():
    """Show how PySpark concepts map to Java MapReduce"""
    print_header("Part 6: Connecting to Java MapReduce")

    print("🔗 Mapping PySpark to Java MapReduce:")
    print("""
Python/PySpark Approach:
spark_df.groupBy("Pclass").agg({"Survived": "avg"})

Java MapReduce Equivalent:
public static class SurvivalMapper extends Mapper<Object, Text, IntWritable, IntWritable> {
    public void map(Object key, Text value, Context context) throws IOException, InterruptedException {
        String[] fields = value.toString().split(",");
        int pclass = Integer.parseInt(fields[2]);  // Pclass column
        int survived = Integer.parseInt(fields[1]); // Survived column
        context.write(new IntWritable(pclass), new IntWritable(survived));
    }
}

public static class SurvivalReducer extends Reducer<IntWritable, IntWritable, IntWritable, FloatWritable> {
    public void reduce(IntWritable key, Iterable<IntWritable> values, Context context) throws IOException, InterruptedException {
        int sum = 0;
        int count = 0;
        for (IntWritable val : values) {
            sum += val.get();
            count++;
        }
        float avg = (float) sum / count;
        context.write(key, new FloatWritable(avg));
    }
}
""")

    print("🎯 This shows how your Java lab implements the same concepts!")
    print("   - Map phase extracts key-value pairs")
    print("   - Shuffle phase groups by key")
    print("   - Reduce phase aggregates values")


## Summary

### Key Takeaways:
1. Big data needs distributed processing
2. MapReduce: map → shuffle → reduce
3. Spark simplifies distributed computing
4. Connects to your Java lab concepts

In [ ]:
def main():
    """Main demo function"""
    print("🚀 Hadoop & MapReduce Student Hands-On Demo")
    print("=" * 50)
    print("Module 10: Scalable Computing - From Single Machine to Distributed Processing")
    print("=" * 50)

    # Check environment
    if not check_environment():
        print("❌ Environment check failed. Please install required packages.")
        return

    # Load data
    df = load_titanic_data()

    # Part 1: Single machine limits
    demonstrate_single_machine_limits(df)

    # Part 2: Manual MapReduce
    manual_results = manual_mapreduce_demo(df)

    # Part 3: PySpark demo
    print("\n⚠️  Note: PySpark may not work in all environments.")
    print("   We'll demonstrate the concepts conceptually instead.")
    spark_results = None  # Skip PySpark for now
    # spark_results = pyspark_demo(df, manual_results)

    # Part 4: Performance comparison (simplified without Spark)
    print_header("Part 4: Performance Analysis")
    print("🐼 Testing Pandas Performance...")
    import time
    start_time = time.time()
    pandas_result = df.groupby('Pclass')['Survived'].agg(['mean', 'count', 'sum'])
    pandas_time = time.time() - start_time
    print(".4f")
    print("\n⚡ Spark comparison skipped due to environment limitations")
    print("   In real Hadoop/Spark clusters, Spark would be much faster on big data!")

    # Part 5: Visualizations
    create_visualizations(df, manual_results, None)  # Skip Spark results

    # Part 6: Java connection
    demonstrate_java_connection()

    # Summary
    print_header("Summary & Key Takeaways")

    print("🎯 What We Learned Today:")
    print("1. The Big Data Problem: Single machines can't handle massive datasets")
    print("2. MapReduce Paradigm: map → shuffle → reduce pattern")
    print("3. PySpark vs Manual: Spark handles complexity automatically")
    print("4. Performance Trade-offs: Different tools for different scales")
    print("5. Real Hadoop Connection: PySpark concepts map to Java MapReduce")

    print("\n🏁 Demo completed successfully!")
    print("💡 Next: Try this with larger datasets to see Spark's real power!")


In [ ]:
if __name__ == "__main__":
    main()